<a href="https://colab.research.google.com/github/ankush2805/EIP/blob/master/Tests5/checked.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# mount gdrive and unzip data
from google.colab import drive
drive.mount('/content/gdrive')
!unzip -q "/content/gdrive/My Drive/hvc_data.zip"
# look for `hvc_annotations.csv` file and `resized` dir
%ls 

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
gdrive/  hvc_annotations.csv  resized/  sample_data/


In [1]:
%tensorflow_version 1.x

import cv2
import json

import numpy as np
import pandas as pd

from functools import partial
from pathlib import Path 
from tqdm import tqdm

from google.colab.patches import cv2_imshow

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder


from keras.applications import VGG16
from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers import Input, GlobalAveragePooling2D
from keras.models import Model
from keras.optimizers import SGD, Adam
from keras.preprocessing.image import ImageDataGenerator


Using TensorFlow backend.


In [3]:
# load annotations
df = pd.read_csv("hvc_annotations.csv")
del df["filename"] # remove unwanted column
df.head()

,gender,imagequality,age,weight,carryingbag,footwear,emotion,bodypose,image_path
0,male,Average,35-45,normal-healthy,Grocery/Home/Plastic Bag,Normal,Neutral,Front-Frontish,resized/1.jpg
1,female,Average,35-45,over-weight,None,Normal,Angry/Serious,Front-Frontish,resized/2.jpg
2,male,Good,45-55,normal-healthy,Grocery/Home/Plastic Bag,CantSee,Neutral,Front-Frontish,resized/3.jpg
3,male,Good,45-55,normal-healthy,Daily/Office/Work Bag,Normal,Neutral,Front-Frontish,resized/4.jpg
4,female,Good,35-45,slightly-overweight,None,CantSee,Neutral,Front-Frontish,resized/5.jpg


In [4]:
# one hot encoding of labels

one_hot_df = pd.concat([
    df[["image_path"]],
    pd.get_dummies(df.gender, prefix="gender"),
    pd.get_dummies(df.imagequality, prefix="imagequality"),
    pd.get_dummies(df.age, prefix="age"),
    pd.get_dummies(df.weight, prefix="weight"),
    pd.get_dummies(df.carryingbag, prefix="carryingbag"),
    pd.get_dummies(df.footwear, prefix="footwear"),
    pd.get_dummies(df.emotion, prefix="emotion"),
    pd.get_dummies(df.bodypose, prefix="bodypose"),
], axis = 1)

one_hot_df.head().T

,0,1,2,3,4
image_path,resized/1.jpg,resized/2.jpg,resized/3.jpg,resized/4.jpg,resized/5.jpg
gender_female,0,1,0,0,1
gender_male,1,0,1,1,0
imagequality_Average,1,1,0,0,0
imagequality_Bad,0,0,0,0,0
imagequality_Good,0,0,1,1,1
age_15-25,0,0,0,0,0
age_25-35,0,0,0,0,0
age_35-45,1,1,0,0,1
age_45-55,0,0,1,1,0


In [0]:
def get_random_eraser(input_img, p=0.5, s_l=0.02, s_h=0.4, r_1=0.3, r_2=1/0.3, v_l=0, v_h=1, pixel_level=False):
  img_h, img_w, img_c = input_img.shape
  p_1 = np.random.rand()
  if p_1 > p:
    return input_img

  while True:
    s = np.random.uniform(s_l, s_h) * img_h * img_w
    r = np.random.uniform(r_1, r_2)
    w = int(np.sqrt(s / r))
    h = int(np.sqrt(s * r))
    left = np.random.randint(0, img_w)
    top = np.random.randint(0, img_h)

    if left + w <= img_w and top + h <= img_h:
      break

    if pixel_level:
      c = np.random.uniform(v_l, v_h, (h, w, img_c))
    else:
      c = np.random.uniform(v_l, v_h)
    input_img[top:top + h, left:left + w, :] = c
  return input_img

In [0]:
def augmentor(images):
		'Apply data augmentation'
		sometimes = lambda aug: iaa.Sometimes(0.5, aug)
		seq = iaa.Sequential(
				[
				# apply the following augmenters to most images
				iaa.Fliplr(0.3),  # horizontally flip 50% of all images
				sometimes(iaa.Affine(
					scale={"x": (0.9, 1.1), "y": (0.9, 1.1)},
					# scale images to 80-120% of their size, individually per axis
					translate_percent={"x": (-0.1, 0.1), "y": (-0.1, 0.1)},
					# translate by -20 to +20 percent (per axis)
					rotate=(-10, 10),  # rotate by -45 to +45 degrees
					shear=(-5, 5),  # shear by -16 to +16 degrees
					order=[0, 1],
					# use nearest neighbour or bilinear interpolation (fast)
					cval=(0, 255),  # if mode is constant, use a cval between 0 and 255
					mode=ia.ALL
					# use any of scikit-image's warping modes (see 2nd image from the top for examples)
				)),
				# execute 0 to 5 of the following (less important) augmenters per image
				# don't execute all of them, as that would often be way too strong
				iaa.SomeOf((0, 5),
				           [sometimes(iaa.Superpixels(p_replace=(0, 1.0),
						                                     n_segments=(20, 200))),
					           # convert images into their superpixel representation
					           iaa.OneOf([
							           iaa.GaussianBlur((0, 1.0)),
							           # blur images with a sigma between 0 and 3.0
							           iaa.AverageBlur(k=(3, 5)),
							           # blur image using local means with kernel sizes between 2 and 7
							           iaa.MedianBlur(k=(3, 5)),
							           # blur image using local medians with kernel sizes between 2 and 7
					           ]),
					           iaa.Sharpen(alpha=(0, 1.0), lightness=(0.9, 1.1)),
					           # sharpen images
					           iaa.Emboss(alpha=(0, 1.0), strength=(0, 2.0)),
					           # emboss images
					           # search either for all edges or for directed edges,
					           # blend the result with the original image using a blobby mask
					           iaa.SimplexNoiseAlpha(iaa.OneOf([
							           iaa.EdgeDetect(alpha=(0.5, 1.0)),
							           iaa.DirectedEdgeDetect(alpha=(0.5, 1.0),
							                                  direction=(0.0, 1.0)),
					           ])),
					           iaa.AdditiveGaussianNoise(loc=0,
					                                     scale=(0.0, 0.01 * 255),
					                                     per_channel=0.5),
					           # add gaussian noise to images
					           
					           
					           iaa.Add((-2, 2), per_channel=0.5),
					           # change brightness of images (by -10 to 10 of original value)
					           iaa.AddToHueAndSaturation((-1, 1)),
					           # change hue and saturation
					           # either change the brightness of the whole image (sometimes
					           # per channel) or change the brightness of subareas
					           iaa.OneOf([
							           iaa.Multiply((0.9, 1.1), per_channel=0.5),
							           iaa.FrequencyNoiseAlpha(
									           exponent=(-1, 0),
									           first=iaa.Multiply((0.9, 1.1),
									                              per_channel=True),
									           second=iaa.ContrastNormalization(
											           (0.9, 1.1))
							           )
					           ]),
					           sometimes(iaa.ElasticTransformation(alpha=(0.5, 3.5),
					                                               sigma=0.25)),
					           # move pixels locally around (with random strengths)
					           sometimes(iaa.PiecewiseAffine(scale=(0.01, 0.05))),
					           # sometimes move parts of the image around
					           sometimes(iaa.PerspectiveTransform(scale=(0.01, 0.1)))
				           ],
				           random_order=True
				           )
				],
				random_order=True
		)
		return seq.augment_images(images)

In [0]:
import cv2
from albumentations import (
    Compose, HorizontalFlip, CLAHE, HueSaturationValue,
    RandomBrightness, RandomContrast, RandomGamma,
    ToFloat, ShiftScaleRotate, Normalize
)

AUGMENTATIONS_TRAIN = Compose([                        
    HorizontalFlip(p=0.5),
    RandomContrast(limit=0.2, p=0.5),
    RandomGamma(gamma_limit=(80, 120), p=0.5),
    RandomBrightness(limit=0.2, p=0.5),
    HueSaturationValue(hue_shift_limit=5, sat_shift_limit=20,
                       val_shift_limit=10, p=.9),
     CLAHE(p=1.0, clip_limit=2.0),
    ShiftScaleRotate(
        shift_limit=0.0625, scale_limit=0.1, 
        rotate_limit=15, border_mode=cv2.BORDER_REFLECT_101, p=0.8), 
    ToFloat(max_value=255)
])

AUGMENTATIONS_TEST = Compose([
    ToFloat(max_value=255)
])

AUGMENTATIONS_NORMALIZE = Compose([
    ToFloat(max_value=255)
])

In [2]:
import keras
import numpy as np
import imgaug as ia
import imgaug.augmenters as iaa

ia.seed(1)

# Label columns per attribute
_gender_cols_ = [col for col in one_hot_df.columns if col.startswith("gender")]
_imagequality_cols_ = [col for col in one_hot_df.columns if col.startswith("imagequality")]
_age_cols_ = [col for col in one_hot_df.columns if col.startswith("age")]
_weight_cols_ = [col for col in one_hot_df.columns if col.startswith("weight")]
_carryingbag_cols_ = [col for col in one_hot_df.columns if col.startswith("carryingbag")]
_footwear_cols_ = [col for col in one_hot_df.columns if col.startswith("footwear")]
_emotion_cols_ = [col for col in one_hot_df.columns if col.startswith("emotion")]
_bodypose_cols_ = [col for col in one_hot_df.columns if col.startswith("bodypose")]

class PersonDataGenerator(keras.utils.Sequence):
    """Ground truth data generator"""

    
    def __init__(self, df, batch_size=32, shuffle=True, augmentations=AUGMENTATIONS_TEST):
        self.df = df
        self.batch_size=batch_size
        self.shuffle = shuffle
        self.on_epoch_end()
        self.augment = augmentations

    def __len__(self):
        return int(np.floor(self.df.shape[0] / self.batch_size))

    def __getitem__(self, index):
        """fetch batched images and targets"""
        batch_slice = slice(index * self.batch_size, (index + 1) * self.batch_size)
        items = self.df.iloc[batch_slice]
        image = np.stack([cv2.imread(item["image_path"]) for _, item in items.iterrows()])

        image = np.stack([AUGMENTATIONS_NORMALIZE(image=img)["image"] for img in image])

        if self.batch_size ==32:
          image = np.stack([get_random_eraser(img) for img in image])
          image = augmentor(image)
        target = {
            "gender_output": items[_gender_cols_].values,
            "image_quality_output": items[_imagequality_cols_].values,
            "age_output": items[_age_cols_].values,
            "weight_output": items[_weight_cols_].values,
            "bag_output": items[_carryingbag_cols_].values,
            "pose_output": items[_bodypose_cols_].values,
            "footwear_output": items[_footwear_cols_].values,
            "emotion_output": items[_emotion_cols_].values,
        }
        return image, target

    def on_epoch_end(self):
        """Updates indexes after each epoch"""
        if self.shuffle == True:
            self.df = self.df.sample(frac=1).reset_index(drop=True)


Using TensorFlow backend.


NameError: ignored

In [20]:
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(one_hot_df, test_size=0.15)
train_df.shape, val_df.shape

((11537, 28), (2036, 28))

In [21]:
train_df.head()

,image_path,gender_female,gender_male,imagequality_Average,imagequality_Bad,imagequality_Good,age_15-25,age_25-35,age_35-45,age_45-55,age_55+,weight_normal-healthy,weight_over-weight,weight_slightly-overweight,weight_underweight,carryingbag_Daily/Office/Work Bag,carryingbag_Grocery/Home/Plastic Bag,carryingbag_None,footwear_CantSee,footwear_Fancy,footwear_Normal,emotion_Angry/Serious,emotion_Happy,emotion_Neutral,emotion_Sad,bodypose_Back,bodypose_Front-Frontish,bodypose_Side
10204,resized/10206.jpg,0,1,0,1,0,0,1,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,1,0,0,1,0
942,resized/943.jpg,1,0,1,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,1,0,0,1,0,0,0,1,0
2491,resized/2492.jpg,0,1,0,0,1,0,1,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,0,1
4229,resized/4230.jpg,1,0,0,0,1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,1,0,1,0,0
5589,resized/5590.jpg,1,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,1,0,0


In [0]:
# create train and validation data generators
train_gen = PersonDataGenerator(train_df, batch_size=32)
valid_gen = PersonDataGenerator(val_df, batch_size=64, shuffle=False)

In [23]:
# get number of output units from data
images, targets = next(iter(train_gen))
num_units = { k.split("_output")[0]:v.shape[1] for k, v in targets.items()}
num_units

{'age': 5,
 'bag': 3,
 'emotion': 4,
 'footwear': 3,
 'gender': 2,
 'image_quality': 3,
 'pose': 3,
 'weight': 4}

In [0]:
def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 0.001
    if epoch > 95:
        lr = 0.000001
    if epoch > 80:
        lr = 0.00005
    elif epoch > 60:
        lr = 0.0001
    elif epoch > 40:
        lr = 0.0005
    elif epoch > 20:
        lr = 0.001
    print('Learning rate: ', lr)
    return lr

In [25]:
from keras.applications import ResNet101V2, ResNet50, InceptionV3, ResNet152V2, InceptionResNetV2, DenseNet201
backbone = ResNet152V2(
    weights=None, 
    include_top=False,
    input_tensor=Input(shape=(224, 224, 3))
)

neck = backbone.output
#neck = Flatten(name="flatten")(neck)
neck = GlobalAveragePooling2D()(neck)
neck = Dense(512, activation="relu")(neck)

def build_dense_tower(in_layer):
    neck = Dense(128, activation="relu")(in_layer)
    neck = Dense(128, activation="relu")(neck)
    neck = Dense(128, activation="relu")(neck)
    neck = Dense(128, activation="relu")(neck)
    return neck

def build_tower(in_layer):
    #neck = Dropout(0.2)(in_layer)
    neck = Dense(128, activation="relu")(in_layer)
    #neck = Dropout(0.3)(in_layer)
    neck = Dense(128, activation="relu")(neck)
    return neck


def build_head(name, in_layer):
    return  Dense(
        num_units[name], activation="softmax", name=f"{name}_output"
    )(in_layer)

# heads
gender = build_head("gender", build_dense_tower(neck))
image_quality = build_head("image_quality", build_dense_tower(neck))
age = build_head("age", build_tower(neck))
weight = build_head("weight", build_tower(neck))
bag = build_head("bag", build_dense_tower(neck))
footwear = build_head("footwear", build_dense_tower(neck))
emotion = build_head("emotion", build_tower(neck))
pose = build_head("pose", build_dense_tower(neck))


model = Model(
    inputs=backbone.input, 
    outputs=[gender, image_quality, age, weight, bag, footwear, pose, emotion]
)
#from keras.models import load_model
#model = load_model('/content/gdrive/My Drive/model_011.hdf5')

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [26]:
# losses = {
# 	"gender_output": "binary_crossentropy",
# 	"image_quality_output": "categorical_crossentropy",
# 	"age_output": "categorical_crossentropy",
# 	"weight_output": "categorical_crossentropy",

# }
# loss_weights = {"gender_output": 1.0, "image_quality_output": 1.0, "age_output": 1.0}
from keras.optimizers import Adam
opt = Adam(lr=lr_schedule(0))
model.compile(
    optimizer=opt,
    loss="categorical_crossentropy", 
    # loss_weights=loss_weights, 
    metrics=["accuracy"]
)

Learning rate:  0.001


In [0]:
def plot_model_history(model_history):
    acc = ['gender_output_acc', 'image_quality_output_acc' , 'age_output_acc', 'weight_output_acc','bag_output_acc', 'footwear_output_acc','pose_output_acc',  'emotion_output_acc']
    val_acc = ['val_gender_output_acc', 'val_image_quality_output_acc' , 'val_age_output_acc', 'val_weight_output_acc','val_bag_output_acc', 'val_footwear_output_acc','val_pose_output_acc',  'val_emotion_output_acc']
    loss = ['gender_output_loss', 'image_quality_output_loss' , 'age_output_loss', 'weight_output_loss','bag_output_loss', 'footwear_output_loss','pose_output_loss',  'emotion_output_loss']
    val_loss = ['val_gender_output_loss', 'val_image_quality_output_loss' , 'val_age_output_loss', 'val_weight_output_loss','val_bag_output_loss', 'val_footwear_output_loss','val_pose_output_loss',  'val_emotion_output_loss']
    fig, axs = plt.subplots(8,2,figsize=(25,25))
    for i in range(8) :
      axs[i][0].plot(range(1,len(model_history.history[acc[i]])+1),model_history.history[acc[i]])
      axs[i][0].plot(range(1,len(model_history.history[val_acc[i]])+1),model_history.history[val_acc[i]])
      axs[i][0].set_title(acc[i])
      
      axs[i][0].set_xticks(np.arange(1,len(model_history.history[val_acc[i]])+1),len(model_history.history[val_acc[i]])/10)
      axs[i][0].legend(['train', 'val'], loc='best')
      # summarize history for loss
      axs[i][1].plot(range(1,len(model_history.history[loss[i]])+1),model_history.history[loss[i]])
      axs[i][1].plot(range(1,len(model_history.history[val_loss[i]])+1),model_history.history[val_loss[i]])
      axs[i][1].set_title(loss[i])
      
      axs[i][1].set_xticks(np.arange(1,len(model_history.history[loss[i]])+1),len(model_history.history[loss[i]])/10)
      axs[i][1].legend(['train', 'val'], loc='best')

    plt.show()

In [28]:
import os
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
#drive.mount('/content/gdrive')
filepath = '/content/gdrive/My Drive/person_attributes_densenet_augment.hdf5'

checkpoint = ModelCheckpoint(filepath=filepath,
                             verbose=1,save_weights_only=False,
                             period =10)

callbacks = [checkpoint]

model_info = model.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=6, 
    epochs=100,
    verbose=1, callbacks=callbacks
)
import matplotlib.pyplot as plt
plot_model_history(model_info)

Epoch 1/100
360/360 [==============================] - 185s 514ms/step - loss: 11.6069 - gender_output_loss: 1.0289 - image_quality_output_loss: 1.4111 - age_output_loss: 2.1085 - weight_output_loss: 1.5888 - bag_output_loss: 1.4081 - footwear_output_loss: 1.5502 - pose_output_loss: 1.2821 - emotion_output_loss: 1.2292 - gender_output_acc: 0.5457 - image_quality_output_acc: 0.5400 - age_output_acc: 0.3822 - weight_output_acc: 0.6161 - bag_output_acc: 0.5460 - footwear_output_acc: 0.4734 - pose_output_acc: 0.6082 - emotion_output_acc: 0.7082 - val_loss: 60.1515 - val_gender_output_loss: 6.8695 - val_image_quality_output_loss: 8.6800 - val_age_output_loss: 12.1887 - val_weight_output_loss: 5.7568 - val_bag_output_loss: 7.2686 - val_footwear_output_loss: 8.8863 - val_pose_output_loss: 6.0017 - val_emotion_output_loss: 4.5000 - val_gender_output_acc: 0.5706 - val_image_quality_output_acc: 0.4582 - val_age_output_acc: 0.2399 - val_weight_output_acc: 0.6411 - val_bag_output_acc: 0.5479 - val

Process ForkPoolWorker-791:
Process ForkPoolWorker-789:
Process ForkPoolWorker-795:
Process ForkPoolWorker-794:
Process ForkPoolWorker-792:
Process ForkPoolWorker-788:
Process ForkPoolWorker-787:
Process ForkPoolWorker-786:
Process ForkPoolWorker-796:
Process ForkPoolWorker-790:
Process ForkPoolWorker-793:
Traceback (most recent call last):
Process ForkPoolWorker-797:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.

KeyboardInterrupt: ignored

In [0]:
model